In [7]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [ ]:
import pandas as pd
data = pd.read_csv(r"C:\Users\jeni7\OneDrive\Documents\CTS_Hackathon\dataset\dataset.csv")
print(data.columns.tolist())
print(data.head())


['age', 'time_in_hospital', 'n_lab_procedures', 'n_procedures', 'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test', 'A1Ctest', 'change', 'diabetes_med', 'readmitted']
       age  time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
0  [70-80)                 8                72             1             18   
1  [70-80)                 3                34             2             13   
2  [50-60)                 5                45             0             18   
3  [70-80)                 2                36             0             12   
4  [60-70)                 1                42             0              7   

   n_outpatient  n_inpatient  n_emergency medical_specialty       diag_1  \
0             2            0            0           Missing  Circulatory   
1             0            0            0             Other        Other   
2             0            0            0           

In [30]:
data['readmitted'] = data['readmitted'].map({'no': 0, 'yes': 1})


In [31]:
X = data.drop(columns=['readmitted'])
y = data['readmitted']


In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = ['age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 
                    'glucose_test', 'A1Ctest', 'change', 'diabetes_med']

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
catboost = CatBoostClassifier(verbose=0, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
lgbm = LGBMClassifier(random_state=42)
ensemble = VotingClassifier(
    estimators=[('cat', catboost), ('xgb', xgb), ('lgbm', lgbm)],
    voting='soft')
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Initial Ensemble Accuracy: {accuracy * 100:.2f}%")


c:\Users\jeni7\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:52:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 9403, number of negative: 10597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001366 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 283
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.470150 -> initscore=-0.119542
[LightGBM] [Info] Start training from score -0.119542
Initial Ensemble Accuracy: 60.92%


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

hsp_df = pd.read_csv(r"C:\Users\jeni7\OneDrive\Documents\CTS_Hackathon\dataset\dataset.csv")
hsp_df['change'] = hsp_df['change'].map({'yes': 1, 'no': 0})
hsp_df['glucose_test'] = hsp_df['glucose_test'].map({'normal': 1, 'high': 2, 'no': 0})
hsp_df['A1Ctest'] = hsp_df['A1Ctest'].map({'normal': 1, 'high': 2, 'no': 0})
hsp_df['diabetes_med'] = hsp_df['diabetes_med'].map({'yes': 1, 'no': 0})

hsp_df['age'] = hsp_df['age'].str.extract('(\d+)').astype(float)  

categorical_cols = hsp_df.select_dtypes(include=['object', 'category']).columns
print("Categorical Columns:", categorical_cols)

for col in categorical_cols:
    le = LabelEncoder()
    hsp_df[col] = le.fit_transform(hsp_df[col].astype(str))

print(hsp_df.head())


<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jeni7\AppData\Local\Temp\ipykernel_23832\670692117.py:15: SyntaxWarning: invalid escape sequence '\d'
  hsp_df['age'] = hsp_df['age'].str.extract('(\d+)').astype(float)  # extracts lower bound


Categorical Columns: Index(['medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'readmitted'], dtype='object')
    age  time_in_hospital  n_lab_procedures  n_procedures  n_medications  \
0  70.0                 8                72             1             18   
1  70.0                 3                34             2             13   
2  50.0                 5                45             0             18   
3  70.0                 2                36             0             12   
4  60.0                 1                42             0              7   

   n_outpatient  n_inpatient  n_emergency  medical_specialty  diag_1  diag_2  \
0             2            0            0                  4       0       7   
1             0            0            0                  5       6       6   
2             0            0            0                  4       0       0   
3             1            0            0                  4       0       6   
4             0            0     

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb

hsp_df = pd.read_csv(r"C:\Users\jeni7\OneDrive\Documents\CTS_Hackathon\dataset\dataset.csv")
hsp_df['change'] = hsp_df['change'].map({'yes': 1, 'no': 0})
hsp_df['glucose_test'] = hsp_df['glucose_test'].map({'normal': 1, 'high': 2, 'no': 0})
hsp_df['A1Ctest'] = hsp_df['A1Ctest'].map({'normal': 1, 'high': 2, 'no': 0})
hsp_df['diabetes_med'] = hsp_df['diabetes_med'].map({'yes': 1, 'no': 0})

hsp_df['age'] = hsp_df['age'].str.extract('(\d+)').astype(float) 

y = hsp_df['readmitted'].map({'no': 0, 'yes': 1})
X = hsp_df.drop('readmitted', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

cat_features = ['medical_specialty','diag_1','diag_2','diag_3']
num_features = [c for c in X_train.columns if c not in cat_features]

for col in cat_features:
    X_train[col] = X_train[col].astype(str).fillna('Missing')
    X_valid[col] = X_valid[col].astype(str).fillna('Missing')

for col in num_features:
    X_train[col] = X_train[col].fillna(X_train[col].median())
    X_valid[col] = X_valid[col].fillna(X_train[col].median())

X_train['total_visits'] = X_train['n_outpatient'] + X_train['n_inpatient'] + X_train['n_emergency']
X_valid['total_visits'] = X_valid['n_outpatient'] + X_valid['n_inpatient'] + X_valid['n_emergency']

X_train['total_procedures'] = X_train['n_lab_procedures'] + X_train['n_procedures'] + X_train['n_medications']
X_valid['total_procedures'] = X_valid['n_lab_procedures'] + X_valid['n_procedures'] + X_valid['n_medications']

X_train['medications_per_day'] = X_train['n_medications'] / X_train['time_in_hospital']
X_valid['medications_per_day'] = X_valid['n_medications'] / X_valid['time_in_hospital']

X_train_enc = X_train.copy()
X_valid_enc = X_valid.copy()
label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X_train_enc[col] = le.fit_transform(X_train_enc[col])
    X_valid_enc[col] = le.transform(X_valid_enc[col])
    label_encoders[col] = le

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_enc, y_train)

cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='Accuracy',
    early_stopping_rounds=50,
    random_seed=42,
    verbose=100
)
cat_model.fit(
    X_train, y_train,
    cat_features=cat_feature_indices,
    eval_set=(X_valid, y_valid),
    use_best_model=True
)

xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=6, eval_metric='logloss', use_label_encoder=False)
xgb_model.fit(X_train_res, y_train_res)

lgb_model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.05, max_depth=6)
lgb_model.fit(X_train_res, y_train_res)

cat_preds = cat_model.predict_proba(X_valid)[:,1]
xgb_preds = xgb_model.predict_proba(X_valid_enc)[:,1]
lgb_preds = lgb_model.predict_proba(X_valid_enc)[:,1]

ensemble_preds = 0.4*cat_preds + 0.3*xgb_preds + 0.3*lgb_preds

best_acc = 0
best_thresh = 0.5
for thresh in np.arange(0.4, 0.61, 0.01):
    preds_bin = (ensemble_preds >= thresh).astype(int)
    acc = accuracy_score(y_valid, preds_bin)
    if acc > best_acc:
        best_acc = acc
        best_thresh = thresh

print(f"Best threshold: {best_thresh:.2f}, Accuracy: {best_acc*100:.2f}%")


<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:21: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jeni7\AppData\Local\Temp\ipykernel_23832\817308942.py:21: SyntaxWarning: invalid escape sequence '\d'
  hsp_df['age'] = hsp_df['age'].str.extract('(\d+)').astype(float)  # lower bound of age range


0:	learn: 0.6086500	test: 0.6062000	best: 0.6062000 (0)	total: 216ms	remaining: 3m 35s
100:	learn: 0.6315500	test: 0.6200000	best: 0.6214000 (91)	total: 9.87s	remaining: 1m 27s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6214
bestIteration = 91

Shrink model to first 92 iterations.


c:\Users\jeni7\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:19:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 10597, number of negative: 10597
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 691
[LightGBM] [Info] Number of data points in the train set: 21194, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi